In [1]:
%load_ext autoreload
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" # 'last' 'none' ...

In [2]:
import pymrio
from pymrio.core.mriosystem import IOSystem as IOS
from pymrio.core.mriosystem import concate_extension
from pymrio.tools.iomath import div0
from pymrio.tools.iofunctions import *
from pymrio.tools.ioparser import *
import pandas as pd
import numpy as np
# import scipy.io
import scipy.sparse as sp 
from scipy.sparse import linalg as spla
import operator
import warnings
import matplotlib.pyplot as plt
from pylab import *
import pickle
from openpyxl import load_workbook
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" # 'last' 'none' ...
warnings.filterwarnings('ignore')

In [3]:
path_io = '/mnt/dd/adrien/DD/Économie/Données/'
path_dl = '/home/adrien/Downloads/'

In [4]:
# exio3 = parse_exiobase3(path_io+'Exiobase 3.4/exiobase3.4_iot_2011_pxp.zip')
# exio3.calc_all()
# exio3.x = exio3.x.squeeze()
# exio3.impact = exio3.satellite

In [5]:
exio3 = pickle.load(open(path_io+'Exiobase 3.4/exio3.pkl', 'rb'))

In [42]:
agg_exio3 = pickle.load(open(path_io+'Exiobase 3.4/agg_exio3.pkl', 'rb'))

In [4]:
def consumption_sectors(self, notion): # TODO: other sectors, difference from elecs_names to add after elec and to display
    '''
    Returns the list of sectors (or names) corresponding to notion, which can be: food, drug, cloth, housing, furniture, health, transport, communication, leisure, education, catering, diverse
    '''
    if notion is None or notion == 'total': sectors = self.sectors
    elif notion not in ['food', 'tobacco', 'cloth', 'housing', 'furniture', 'health', 'transport', 'communication', 'leisure', 'education', 'catering', 'diverse']:
        print('Error: notion unknown')
    elif notion=='food': sectors = ['Paddy rice',  'Wheat',  'Cereal grains nec',  'Vegetables, fruit, nuts',  'Oil seeds',  'Sugar cane, sugar beet',  'Plant-based fibers',  'Crops nec',  'Cattle',  'Pigs',  'Poultry',  'Meat animals nec',  'Animal products nec',  'Raw milk', 'Fish and other fishing products; services incidental of fishing (05)', 
'Products of meat cattle', 'Products of meat pigs', 'Products of meat poultry', 'Meat products nec', 'products of Vegetable oils and fats', 'Dairy products', 'Processed rice', 'Sugar', 'Food products nec', 'Beverages', 'Fish products']
    elif notion=='tobacco': sectors = ['Tobacco products (16)']
    elif notion=='cloth': sectors = ['Textiles (17)', 'Wearing apparel; furs (18)', 'Leather and leather products (19)']
    elif notion=='housing': sectors = ['Electricity by coal', 'Electricity by gas', 'Electricity by nuclear', 'Electricity by hydro', 'Electricity by wind', 'Electricity by petroleum and other oil derivatives', 'Electricity by biomass and waste', 'Electricity by solar photovoltaic', 'Electricity by solar thermal', 'Electricity by tide, wave, ocean', 'Electricity by Geothermal', 'Electricity nec', 'Steam and hot water supply services', 'Collected and purified water, distribution services of water (41)', 'Construction work (45)', 'Real estate services (70)']
    elif notion=='furniture': sectors = ['Paper and paper products', 'Printed matter and recorded media (22)', 'Furniture; other manufactured goods n.e.c. (36)'] 
    elif notion=='health': sectors = ['Health and social work services (85)'] 
    elif notion=='transport': sectors = ['Motor Gasoline', 'Gas/Diesel Oil', 'Heavy Fuel Oil', 'Refinery Gas', 'Liquefied Petroleum Gases (LPG)', 'Kerosene', 'Motor vehicles, trailers and semi-trailers (34)', 'Other transport equipment (35)', 'Sale, maintenance, repair of motor vehicles, motor vehicles parts, motorcycles, motor cycles parts and accessoiries', 'Retail trade services of motor fuel', 'Railway transportation services', 'Other land transportation services', 'Transportation services via pipelines', 'Sea and coastal water transportation services', 'Inland water transportation services', 'Air transport services (62)', 'Supporting and auxiliary transport services; travel agency services (63)'] 
    elif notion=='communication': sectors = ['Office machinery and computers (30)', 'Electrical machinery and apparatus n.e.c. (31)', 'Radio, television and communication equipment and apparatus (32)', 'Post and telecommunication services (64)'] 
    elif notion=='leisure': sectors = ['Recreational, cultural and sporting services (92)'] 
    elif notion=='education': sectors = ['Education services (80)'] 
    elif notion=='catering': sectors = ['Hotel and restaurant services (55)'] 
    elif notion=='diverse': sectors = ['Fabricated metal products, except machinery and equipment (28)', 'Machinery and equipment n.e.c. (29)', 'Wholesale trade and commission trade services, except of motor vehicles and motorcycles (51)', 'Retail  trade services, except of motor vehicles and motorcycles; repair services of personal and household goods (52)', 
'Wool, silk-worm cocoons',  'Manure (conventional treatment)',  'Manure (biogas treatment)',  'Products of forestry, logging and related services (02)', 'Anthracite',  'Coking Coal',  'Other Bituminous Coal',  'Sub-Bituminous Coal',  'Patent Fuel',  'Lignite/Brown Coal',  'BKB/Peat Briquettes',  'Peat', 'Natural Gas Liquids', 'Other Hydrocarbons', 'Uranium and thorium ores (12)', 'Iron ores', 'Copper ores and concentrates', 'Nickel ores and concentrates', 'Aluminium ores and concentrates', 'Precious metal ores and concentrates', 'Lead, zinc and tin ores and concentrates', 'Other non-ferrous metal ores and concentrates', 'Stone', 'Sand and clay', 'Chemical and fertilizer minerals, salt and other mining and quarrying products n.e.c.', 'Wood and products of wood and cork (except furniture); articles of straw and plaiting materials (20)', 'Wood material for treatment, Re-processing of secondary wood material into new wood material', 'Pulp', 'Secondary paper for treatment, Re-processing of secondary paper into new pulp', 'Crude petroleum and services related to crude oil extraction, excluding surveying', 'Natural gas and services related to natural gas extraction, excluding surveying', 'Coke Oven Coke', 'Gas Coke', 'Coal Tar', 'Aviation Gasoline', 'Gasoline Type Jet Fuel', 'Kerosene Type Jet Fuel', 'Refinery Feedstocks', 'Ethane', 'Naphtha', 'White Spirit & SBP', 'Lubricants', 'Bitumen', 'Paraffin Waxes', 'Petroleum Coke', 'Non-specified Petroleum Products', 'Nuclear fuel', 'Plastics, basic', 'Secondary plastic for treatment, Re-processing of secondary plastic into new plastic', 'N-fertiliser', 'P- and other fertiliser', 'Chemicals nec', 'Charcoal', 'Additives/Blending Components', 'Biogasoline', 'Biodiesels', 'Other Liquid Biofuels', 'Rubber and plastic products (25)', 'Glass and glass products', 'Secondary glass for treatment, Re-processing of secondary glass into new glass', 'Ceramic goods', 'Bricks, tiles and construction products, in baked clay', 'Cement, lime and plaster', 'Ash for treatment, Re-processing of ash into clinker', 'Other non-metallic mineral products', 'Basic iron and steel and of ferro-alloys and first products thereof', 'Secondary steel for treatment, Re-processing of secondary steel into new steel', 'Precious metals', 'Secondary preciuos metals for treatment, Re-processing of secondary preciuos metals into new preciuos metals', 'Aluminium and aluminium products', 'Secondary aluminium for treatment, Re-processing of secondary aluminium into new aluminium', 'Lead, zinc and tin and products thereof', 'Secondary lead for treatment, Re-processing of secondary lead into new lead', 'Copper products', 'Secondary copper for treatment, Re-processing of secondary copper into new copper', 'Other non-ferrous metal products', 'Secondary other non-ferrous metals for treatment, Re-processing of secondary other non-ferrous metals into new other non-ferrous metals', 'Foundry work services', 'Medical, precision and optical instruments, watches and clocks (33)', 'Secondary raw materials', 'Bottles for treatment, Recycling of bottles by direct reuse', 'Transmission services of electricity', 'Distribution and trade services of electricity', 'Coke oven gas', 'Blast Furnace Gas', 'Oxygen Steel Furnace Gas', 'Gas Works Gas', 'Biogas', 'Distribution services of gaseous fuels through mains', 'Secondary construction material for treatment, Re-processing of secondary construction material into aggregates', 'Financial intermediation services, except insurance and pension funding services (65)', 'Insurance and pension funding services, except compulsory social security services (66)', 'Services auxiliary to financial intermediation (67)', 'Renting services of machinery and equipment without operator and of personal and household goods (71)', 'Computer and related services (72)', 'Research and development services (73)', 'Other business services (74)', 'Public administration and defence services; compulsory social security services (75)', 'Food waste for treatment: incineration', 'Paper waste for treatment: incineration', 'Plastic waste for treatment: incineration', 'Intert/metal waste for treatment: incineration', 'Textiles waste for treatment: incineration', 'Wood waste for treatment: incineration', 'Oil/hazardous waste for treatment: incineration', 'Food waste for treatment: biogasification and land application', 'Paper waste for treatment: biogasification and land application', 'Sewage sludge for treatment: biogasification and land application', 'Food waste for treatment: composting and land application', 'Paper and wood waste for treatment: composting and land application', 'Food waste for treatment: waste water treatment', 'Other waste for treatment: waste water treatment', 'Food waste for treatment: landfill', 'Paper for treatment: landfill', 'Plastic waste for treatment: landfill', 'Inert/metal/hazardous waste for treatment: landfill', 'Textiles waste for treatment: landfill', 'Wood waste for treatment: landfill', 'Membership organisation services n.e.c. (91)', 'Other services (93)', 'Private households with employed persons (95)', 'Extra-territorial organizations and bodies'] 
    elif notion=='undecided': sectors = ['Beverages', 'Paper and paper products', 'Printed matter and recorded media (22)']
    return(sectors)

IOS.consumption_sectors = consumption_sectors

In [6]:
sectors = ['food', 'tobacco', 'cloth', 'housing', 'furniture', 'health', 'transport', 'communication', 'leisure', 'education', 'catering', 'diverse']
imp = exio3.impact.F.index

In [7]:
agg_region = [1 if x=='FR' else 0 for x in exio3.regions]
agg_sector = []
for s in exio3.sectors:
    for i, sec in enumerate(sectors):  
        if s in exio3.consumption_sectors(sec):
            agg_sector = agg_sector + [i]

In [8]:
agg_exio3 = exio3.aggregate(region_agg = agg_region, sector_agg = agg_sector, region_names = ['RoW', 'FR'], sector_names = sectors, inplace = False) 
agg_exio3.calc_all()
agg_exio3.x = agg_exio3.x.iloc[:,0]

In [9]:
agg_impacts = agg_exio3.impact.F
agg_exio3.impact.F = agg_impacts.append([ # TODO: énergie, déchets, minerais, bois TODO: Global Warming Potential (CO2eq)
#     pd.Series(agg_impacts.iloc[['CO2 - combustion - air' in i for i in imp],:].sum(axis = 0), name = 'CO2 - combustion - air (kg)'),
    pd.Series(agg_impacts.iloc[['CO2' in i for i in imp],:].sum(axis = 0), name = 'CO2 (kg)'),
    pd.Series(agg_impacts.iloc[['CH4' in i for i in imp],:].sum(axis = 0), name = 'CH4 (kg)'),
    pd.Series(agg_impacts.iloc[['NOx' in i for i in imp],:].sum(axis = 0), name = 'NOx (kg)'),
    pd.Series(agg_impacts.iloc[['SOx' in i for i in imp],:].sum(axis = 0), name = 'SOx (kg)'),
    pd.Series(agg_impacts.iloc[['N2O' in i for i in imp],:].sum(axis = 0), name = 'N2O (kg)'),
    pd.Series(agg_impacts.iloc[['NH3' in i for i in imp],:].sum(axis = 0), name = 'NH3 (kg)'),
    pd.Series(agg_impacts.iloc[['CO -' in i for i in imp],:].sum(axis = 0), name = 'CO (kg)'),
    pd.Series(agg_impacts.iloc[['PM10' in i for i in imp],:].sum(axis = 0), name = 'PM10 (kg)'),
    pd.Series(agg_impacts.iloc[['PM2.5' in i for i in imp],:].sum(axis = 0), name = 'PM2.5 (kg)'),
    pd.Series(agg_impacts.iloc[['Pb' in i for i in imp],:].sum(axis = 0), name = 'Pb (kg)'),
    pd.Series(agg_impacts.iloc[['Cd' in i for i in imp],:].sum(axis = 0), name = 'Cd (kg)'),
    pd.Series(agg_impacts.iloc[['Cropland' in i for i in imp],:].sum(axis = 0), name = 'Cropland (km2)'),
    pd.Series(agg_impacts.iloc[['Forest area' in i for i in imp],:].sum(axis = 0), name = 'Forest area (km2)'),
    pd.Series(agg_impacts.iloc[['Permanent pasture' in i for i in imp],:].sum(axis = 0), name = 'Permanent pastures (km2)'),
    pd.Series(agg_impacts.iloc[['Cropland' in i or 'land Use' in i or 'pasture' in i for i in imp],:].sum(axis = 0), name = 'Land use (km2)'),
    pd.Series(agg_impacts.iloc[['Water Consumption Blue' in i for i in imp],:].sum(axis = 0), name = 'Water Consumption Blue (Mm3)'),
    pd.Series(agg_impacts.iloc[['Water Consumption Green' in i for i in imp],:].sum(axis = 0), name = 'Water Consumption Green (Mm3)'),
    pd.Series(agg_impacts.iloc[['Water Withdrawal' in i for i in imp],:].sum(axis = 0), name = 'Water Withdrawal Blue (Mm3)'),
    pd.Series(agg_impacts.iloc[['wages' in i for i in imp],:].sum(0), name='Wages (incl. social contributions) (M€)'),
    pd.Series(agg_impacts.iloc[['surplus' in i for i in imp],:].sum(0), name='Operating surplus (investment, rents, profit) (M€)'),
    pd.Series(agg_impacts.iloc[['axes' in i or 'wages' in i or 'surplus' in i for i in imp],:].sum(0), name='Value added (wages, surplus and taxes) (M€)'),
    pd.Series(agg_impacts.iloc[['Employment:' in i and 'Vulnerable' not in i for i in imp],:].sum(0), name='Employment (k pers)'),
    pd.Series(agg_impacts.iloc[['Employment hours:' in i and 'Vulnerable' not in i for i in imp],:].sum(0), name = 'Employment hours (M.hr)')])
del(agg_impacts)
agg_exio3.impact.F = agg_exio3.impact.F.drop(agg_exio3.impact.F.index[range(len(imp))])

In [31]:
pd.set_option('display.max_rows', 25)
agg_exio3.impact.F

In [98]:
# pd.set_option('display.max_rows', 2000)
# exio3.impact.unit

In [14]:
impacts = ['CO2 (kg)', 'CH4 (kg)', 'NOx (kg)', 'SOx (kg)', 'N2O (kg)', 'NH3 (kg)', 'CO (kg)', 'PM10 (kg)', 'PM2.5 (kg)', 'Pb (kg)', 'Cd (kg)', 
          'Cropland (km2)', 'Forest area (km2)', 'Permanent pastures (km2)', 'Land use (km2)', 'Water Consumption Blue (Mm3)', 'Water Consumption Green (Mm3)',
          'Water Withdrawal Blue (Mm3)', 'Wages (incl. social contributions) (M€)', 'Operating surplus (investment, rents, profit) (M€)', 
          'Value added (wages, surplus and taxes) (M€)', 'Employment (k pers)', 'Employment hours (M.hr)']

In [15]:
# /!\ Pb: l'aggrégation fausse les résultats légèrement (par exemple émissions de CO2 sous-estimés de 20%) TODO: faire calculs exacts
agg_impacts_FR = pd.DataFrame(index = sectors, columns = impacts)
for impact in impacts:
    for secs in sectors:
        agg_impacts_FR.at[secs, impact] = agg_exio3.embodied_impact(regs = 'FR', secs = secs, var = impact).sum()
agg_impacts_FR = agg_impacts_FR.append(agg_impacts_FR.sum().rename('Total'))

In [24]:
agg_impacts_FR

,CO2 (kg),CH4 (kg),NOx (kg),SOx (kg),N2O (kg),NH3 (kg),CO (kg),PM10 (kg),PM2.5 (kg),Pb (kg),...,Permanent pastures (km2),Land use (km2),Water Consumption Blue (Mm3),Water Consumption Green (Mm3),Water Withdrawal Blue (Mm3),Wages (incl. social contributions) (M€),"Operating surplus (investment, rents, profit) (M€)","Value added (wages, surplus and taxes) (M€)",Employment (k pers),Employment hours (M.hr)
food,4.00241e+10,1.12169e+09,1.89376e+08,1.03659e+08,4.90656e+07,5.87098e+08,5.13014e+08,3.64087e+07,2.70592e+07,744064,...,90323,215201,4154.74,42173.9,816.785,84131.4,56882.1,153888,6158.08,12228.1
tobacco,3.16729e+08,1.84708e+06,1.6293e+06,1.08114e+06,36856,489848,2.84504e+06,272779,205852,5414.24,...,87.1604,190.712,4.66532,31.8719,3.86278,470.991,283.446,835.528,29.5249,61.8469
cloth,3.77565e+09,3.04175e+07,1.37394e+07,1.14243e+07,639340,8.58281e+06,4.0446e+07,3.64571e+06,2.57247e+06,88794.8,...,1667.46,3548.86,97.3751,595.556,114.377,8309.02,4643.61,14268.8,459.508,932.862
housing,6.34682e+10,3.68515e+08,2.10842e+08,1.77376e+08,4.57794e+06,7.35069e+07,6.81256e+08,6.01336e+07,4.20319e+07,1.48001e+06,...,10382.5,25637.2,660.991,3628.15,1145.97,150996,97957.2,272023,6023.69,11500.1
furniture,1.23614e+10,8.13996e+07,4.41132e+07,3.7607e+07,1.17255e+06,2.09169e+07,1.35093e+08,1.21917e+07,8.60035e+06,301261,...,2752.17,6448.47,173.626,970.935,258.836,28051.3,15506.8,47635.2,1299.07,2559.7
health,2.16283e+10,1.96348e+08,7.58405e+07,6.3152e+07,4.91169e+06,6.51082e+07,2.30908e+08,2.08434e+07,1.46086e+07,503471,...,10142.5,23833.1,526.532,4216.56,438.209,48026.5,29845.7,84868.4,2236.28,4352.01
transport,7.53308e+10,4.02452e+08,3.44701e+08,2.44172e+08,4.38767e+06,7.6111e+07,7.28628e+08,6.8406e+07,5.05427e+07,1.56573e+06,...,9744.41,23980.2,635.935,3226.34,996.213,128970,73005.9,223262,6431.93,13192
communication,1.58486e+10,1.01902e+08,5.5451e+07,4.73327e+07,1.14894e+06,2.00714e+07,1.78307e+08,1.59056e+07,1.12737e+07,406893,...,2764.27,6761.11,178.532,936.249,278.742,43675,26603.4,76850.9,1614.76,3152.67
leisure,1.19391e+10,9.45693e+07,4.61599e+07,3.20751e+07,2.59077e+06,3.34566e+07,1.16764e+08,1.00392e+07,7.18734e+06,225427,...,5603.27,12651.5,288.578,2258.22,204.11,28882.5,18628.6,51979.2,1232.81,2364.94
education,8.38992e+09,8.18834e+07,3.38379e+07,2.18805e+07,2.72022e+06,3.32782e+07,8.11498e+07,6.8195e+06,4.90359e+06,146072,...,5513.02,12680,269.933,2357.52,138.849,19423,12450.6,34869.7,868.778,1670.65


In [109]:
impacts_FR = pd.DataFrame(index = sectors, columns = ['CO2 - combustion - air'])
for impact in ['CO2 - combustion - air']:
    for secs in sectors:
        impacts_FR.at[secs,impact] = exio3.embodied_impact(regs = 'FR', secs = exio3.consumption_sectors(secs), var = impact).sum() # TODO: units
    #     print(secs, round(exio3.embodied_impact(regs = 'FR', secs = exio3.consumption_sectors(secs), var = 'CO2 - combustion - air').sum() / 10**9), ' MtCO2/an')
impacts_FR = impacts_FR.append(impacts_FR.sum().rename('Total'))

In [ ]:
impacts_FR = pickle.load(open(path_io + 'Exiobase 3.4/impacts_FR.pkl', 'wb'))

In [111]:
(impacts_FR / 10**9).astype(integer)

In [45]:
# dir(exio3)
# list(exio3.x.squeeze() * exio3.is_in(secs = None, regs='FR'))

In [19]:
conso_age = pd.DataFrame(index = sectors, columns = range(100))
for sec in sectors: conso_age.at[sec,:] = pd.read_excel(path_dl + 'conso_age_FR_2011.xlsx', header = None, sheet_name = sec).iloc[:,0]
conso_age = conso_age.div(conso_age.sum(1), axis=0) # at i,j: share of conso of sector i by age j (so that sum of each row = 1)

In [20]:
agg_impacts_age = pd.DataFrame(index = sectors, columns = pd.MultiIndex.from_product([impacts, list(range(100))], names=('impact', 'age')))
for sec in sectors:
    for i in impacts:
        agg_impacts_age.loc[sec, [(i, a) for a in range(100)]] = np.array(conso_age.loc[sec] * agg_impacts_FR.at[sec, i])

In [110]:
agg_impacts_age

In [21]:
with pd.ExcelWriter(path_dl + 'impacts_age_FR_2011.xlsx') as writer:  
    for i in impacts: 
        if len(i) < 32: name = i
        elif i == 'Wages (incl. social contributions) (M€)': name = 'Wages (M€)'
        elif i == 'Operating surplus (investment, rents, profit) (M€)': name = 'Operating surplus (M€)'
        elif i == 'Value added (wages, surplus and taxes) (M€)': name = 'Value added (M€)'
        else: print(i)
        agg_impacts_age[i].to_excel(writer, sheet_name = name) 

In [22]:
pickle.dump(agg_impacts_age, open(path_io + 'Exiobase 3.4/agg_impacts_age.pkl', 'wb'))

In [17]:
pickle.dump(agg_impacts_FR, open(path_io + 'Exiobase 3.4/impacts_FR.pkl', 'wb'))

In [45]:
pickle.dump(impacts_FR, open(path_io + 'Exiobase 3.4/impacts_FR.pkl', 'wb'))

In [ ]:
pickle.dump(agg_exio3, open(path_io + 'Exiobase 3.4/agg_exio3.pkl', 'wb'))

In [13]:
pickle.dump(exio3, open(path_io + 'Exiobase 3.4/exio3.pkl', 'wb'))